**Задание 1.**
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [12]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [17]:
import pandas as pd

In [18]:
def check_region(keyword):
    geo_data = { 'Центр': ['москва', 'тула', 'ярославль'],
                 'Северо-Запад': ['петербург', 'псков', 'мурманск'],
                 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']  }
    for key, values in geo_data.items():
        for value in values:
            if  value in keyword:
                return key
    return 'undefined'

In [58]:
keywords = pd.read_csv('keywords.csv')
keywords['region'] = keywords['keyword'].apply(check_region)

In [59]:
print(keywords.head(15))
print('---------------')
print(keywords[365:375])
print('---------------')
print(keywords[845:860])
print('---------------')
print(keywords.tail(15))

                       keyword     shows     region
0                           вк  64292779  undefined
1                одноклассники  63810309  undefined
2                        порно  41747114  undefined
3                         ютуб  39995567  undefined
4                    вконтакте  21014195  undefined
5   одноклассники моя страница  19976138  undefined
6                         майл  15144673  undefined
7                        авито  14786593  undefined
8                   переводчик  14453469  undefined
9                       яндекс  13344734  undefined
10             сбербанк онлайн  10725673  undefined
11                        mail   7687204  undefined
12                        игры   6075762  undefined
13                     youtube   5729958  undefined
14      вконтакте моя страница   5971451  undefined
---------------
                      keyword   shows        region
365               карта метро  457594     undefined
366                   комедии  432097     undefi

**Задание 2.** Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [56]:
import pandas as pd
import numpy as np

In [57]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

def ratings_cl(row):  
        return pd.Series(['низкий рейтинг' if row['rating'] <= 2 else 'средний рейтинг' if 2 < row['rating'] <= 4 else 'высокий рейтинг'])

ratings['Class'] = ratings.apply(ratings_cl, axis=1)
ratings.head()

,userId,movieId,rating,timestamp,Class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


**Задание 3.**
Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [7]:
import pandas as pd
from datetime import datetime

In [8]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [9]:
ratings_100 = ratings.groupby('userId').count()             
ratings_100 = ratings_100[ratings_100['timestamp'] >= 100].reset_index()
ratings_100.head(5)

,userId,movieId,rating,timestamp
0,1,232,232,232
1,4,216,216,216
2,6,314,314,314
3,7,152,152,152
4,10,140,140,140


In [10]:
ratings = ratings[ratings['userId'].isin(ratings_100['userId'].tolist())]
ratings = ratings.groupby('userId').agg(['min', 'max'])['timestamp']
ratings['lifetime'] = ratings\
            .apply(lambda x: abs(datetime.fromtimestamp(x['max']) - datetime.fromtimestamp(x['min'])).days, axis=1)
ratings.head(20)

,min,max,lifetime
userId,,,
1,964980499,965719662,8
4,945078428,1007574542,723
6,845553109,845556915,0
7,1106635416,1176181731,804
10,1455301553,1455619275,3
15,1299424762,1510578158,2443
17,1305696226,1326943248,245
18,1455049328,1534786448,922
19,965701107,965712882,0


In [11]:
mean_lifetime = ratings.agg(['mean'])['lifetime']
print(mean_lifetime.values[0])

511.06854838709677
